### Gstore Analytics, Data Cleaning with Pandas

Starting to work with some of the data provided for a kaggle competition and cleaning it using pandas.

Competition can be found at:
https://www.kaggle.com/c/ga-customer-revenue-prediction

In [329]:
#Import Packages
import pandas as pd

In [330]:
#Load Data Training Data
df = pd.read_csv('train.csv')

/Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [331]:
#Using a very small slice of the full data frame to test cleaning operations before
#applying them to the full training set
test_df = df[:10].copy()
test_df.head(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386


In [332]:
test_df.shape

(10, 12)

In [333]:
#Drop Duplicate Columns
test_df.drop(columns = ['sessionId','visitId']);

#fixes an error where non capatalized 'false' and 'true' are not recognized by pandas
false = False
true = True

In [334]:
#A few of the columns in the data set contain dictionaries for each element
#This is a function that will expand a 'dictionary column' into a new dataframe

def dict_to_columns(dataframe,column_name):
    df = dataframe.copy()
    df[column_name] = df[column_name].apply(lambda x: dict(eval(x)))
    new_dataframe = df[column_name].apply(pd.Series)
    dataframe.drop(columns = column_name)
    return new_dataframe

dict_names = ['device','geoNetwork','totals','trafficSource']

In [335]:
#Iterate through each of the columns containing dictionary elements and convert them to columns
for name in dict_names:
    dict_df = dict_to_columns(test_df,name)
    test_df = pd.concat([test_df,dict_df],axis = 1)

#Drop dict columns after they have been converted
test_df = test_df.drop(columns = dict_names);

In [336]:
test_df.head(3)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,hits,pageviews,bounces,newVisits,campaign,source,medium,keyword,adwordsClickInfo,isTrueDirect
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,1,1,1,1,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,1,1,1,1,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,1,1,1,1,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN


In [337]:
#many columns simply display "not available in Demo", which means they are of no use and should be dropped

for column_name in test_df.columns:
    if type(test_df[column_name][1]) == str:
        if 'not available' in test_df[column_name][1]:
            test_df = test_df.drop(columns = column_name)

test_df.head(3)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,operatingSystem,...,hits,pageviews,bounces,newVisits,campaign,source,medium,keyword,adwordsClickInfo,isTrueDirect
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,Windows,...,1,1,1,1,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,Macintosh,...,1,1,1,1,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,Windows,...,1,1,1,1,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN


In [338]:
#This column was treated as a dict so it has to be removed manually
test_df = test_df.drop(columns = 'adwordsClickInfo')

In [339]:
test_df.head(5)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,operatingSystem,...,visits,hits,pageviews,bounces,newVisits,campaign,source,medium,keyword,isTrueDirect
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,Windows,...,1,1,1,1,1,(not set),google,organic,(not provided),NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,Macintosh,...,1,1,1,1,1,(not set),google,organic,(not provided),NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,Windows,...,1,1,1,1,1,(not set),google,organic,(not provided),NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,Linux,...,1,1,1,1,1,(not set),google,organic,google + online,NaN
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,Android,...,1,1,1,1,NaN,(not set),google,organic,(not provided),True


Now that the columns have been properly identified the next steps will be converting data types for less memory usage and dealing with some of the missing data values